In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import tensorflow as tf
from tensorflow import keras
import pix2pix
from unet import unet_model

In [ ]:
resnet = tf.keras.applications.ResNet50V2(include_top=False, 
                                         weights='imagenet', 
                                         input_shape=(224,224,3))
detection_model = tf.keras.Sequential([resnet,
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(2, activation=None),
                             tf.keras.layers.Softmax()])
detection_model.save("detection.h5")

In [ ]:
OUTPUT_CHANNELS = 1
base_model = tf.keras.applications.MobileNetV2(input_shape=[512, 512, 3], include_top=False)
# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 512x512
    'block_3_expand_relu',   # 256x256
    'block_6_expand_relu',   # 128x128
    'block_13_expand_relu',  # 64x64
    'block_16_project',      # 32x32
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)
down_stack.trainable = True
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
inputs = tf.keras.layers.Input(shape=[512, 512, 3])
x = inputs

# Downsampling through the model
skips = down_stack(x)
x = skips[-1]
skips = reversed(skips[:-1])

# Upsampling and establishing the skip connections
for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])
# This is the last layer of the model
last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 3, strides=2,padding='same', activation="sigmoid")  #64x64 -> 128x128
x = last(x)
segmentation_model = tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
segmentation_model.save("segmentation.h5")

In [ ]:
segmentation_model.summary()